In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow.keras as tfk

In [ ]:
train_path = 'assets\\train'
valid_path = 'assets\\valid'
test_path = 'assets\\test'

In [ ]:
dataset = pd.read_csv('assets\\WILDCATS.csv')
dataset

In [ ]:
dataset.isna().sum()

In [ ]:
dataset_labels = np.unique(dataset['labels'])
dataset_labels

In [ ]:
len(os.listdir('assets\\train'))

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_path,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
train_dataset

In [ ]:
valid_dataset = tf.keras.utils.image_dataset_from_directory(valid_path,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
valid_dataset

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(10, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

In [ ]:
initial_epochs = 15

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=valid_dataset)

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['sparse_categorical_accuracy'])

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         validation_data=valid_dataset)

In [ ]:
model.save('assets\\model.keras')

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(test_path,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
test_dataset

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

print('Predictions:\n', np.array([predictions[i].argmax() for i in range(len(predictions))]).flatten())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[label_batch[i]])
  plt.axis("off")

In [ ]:
custom_folder = os.listdir('assets\\custom')
custom_img = ['assets\\custom\\' + fname for fname in custom_folder]
load_model = tf.keras.models.load_model(filepath='assets\\model.keras')
cls_names = ['AFRICAN LEOPARD', 'CARACAL', 'CHEETAH', 'CLOUDED LEOPARD',
       'JAGUAR', 'LIONS', 'OCELOT', 'PUMA', 'SNOW LEOPARD', 'TIGER']
un_cls_names = np.unique(cls_names)

In [ ]:
def pred_img(img):
    plt.imshow(plt.imread(img))
    img = tf.image.resize(tf.constant(plt.imread(img)), size=(128,128))
    img = np.expand_dims(img, axis=0)
    pred = load_model.predict(img)
    pred_num = pred.argmax()
    pred_per = (pred.max()*100).round(2)   
    plt.axis('off')
    plt.title(f'{pred_per}% {cls_names[pred_num]}')

In [ ]:
def plot_preds(img):
    img = tf.image.resize(tf.constant(plt.imread(img)), size=(128,128))
    img = np.expand_dims(img, axis=0)
    pred = load_model.predict(img)
    plt.bar(np.arange(len(cls_names)), pred[0][pred[0].argsort()[::-1]])
    plt.xticks(np.arange(len(cls_names)), un_cls_names[pred[0].argsort()[::-1]], rotation="vertical", fontsize=8);

In [ ]:
def img_and_pred(img_num=0): 
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    pred_img(custom_img[img_num])
    plt.subplot(1, 2, 2)
    plot_preds(custom_img[img_num])
    plt.tight_layout();

In [ ]:
img_and_pred(1)